In [2]:
psw = 'Tonkawes13-'

In [3]:
import mysql.connector

In [4]:
cnx = mysql.connector.connect(user = 'root',
                              password = psw,
                              host= 'localhost',
                              database = 'sakila',
                              auth_plugin = 'mysql_native_password')

In [5]:
cnx.is_connected()

True

In [6]:
import pandas as pd

def rentals_month(engine, month, year):
    cursor = engine.cursor()
    
    query = f"""
    SELECT *
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    
    cursor.execute(query) 
    result = cursor.fetchall()
    cursor.close()   
    
    rental_data = pd.DataFrame(result, columns = [head[0] for head in cursor.description])
    

    return rental_data

May = rentals_month(cnx, 5, 2005)
June = rentals_month(cnx, 6, 2005)

In [9]:
def rental_count_months (df):
    return df.groupby('customer_id').count()[['rental_id']]

count_june = rental_count_months(June)
count_may = rental_count_months(May)
count_may

,rental_id
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [8]:
def rental_count_month(df1, df2):
    joined_df = df1.add_suffix('_count_month1').join(df2.add_suffix('_count_month2'))
    joined_df['difference'] = joined_df['rental_id_count_month1']-joined_df['rental_id_count_month2']
    return joined_df

rental_count_month(count_may, count_june)

,rental_id_count_month1,rental_id_count_month2,difference
customer_id,,,
1,2,7.0,-5.0
2,1,1.0,0.0
3,2,4.0,-2.0
5,3,5.0,-2.0
6,3,4.0,-1.0
...,...,...,...
594,4,6.0,-2.0
595,1,2.0,-1.0
596,6,2.0,4.0
